In [1]:
%who

Interactive namespace is empty.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings("ignore")

/Users/hanlinzhuang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv('../raw data/new/ltc_hourly.csv')

In [4]:
data['vol_per_count'] = data['volume_traded']/data['trades_count']
data['roi'] = np.log(data['price_close']/data['price_close'].shift())*100
data['price_close_lag1'] = data['price_close'].shift(1)
data['price_close_lag2'] = data['price_close'].shift(2)
data['price_close_lag3'] = data['price_close'].shift(3)
data['volume_traded_lag1'] = data['volume_traded'].shift(1)
data['trades_count_lag1'] = data['trades_count'].shift(1)
data['vol_per_count_lag1'] = data['vol_per_count'].shift(1)
data['roi_lag1'] = data['roi'].shift(1)
data['roi_lag2'] = data['roi'].shift(2)
data['roi_lag3'] = data['roi'].shift(3)

In [5]:
data['roi'].describe()

count    2135.000000
mean        0.038791
std         2.186347
min       -13.526729
25%        -0.685337
50%         0.035736
75%         0.714396
max        14.954936
Name: roi, dtype: float64

In [6]:
std = data['roi'].std()
data['cut'] = pd.cut(data['roi'], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])
data['cut_lag1'] = data['cut'].shift(1)
data['cut_lag2'] = data['cut'].shift(2)
data['cut_lag3'] = data['cut'].shift(3)

In [7]:
data.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
0,2019-02-01T00:00:00.0000000Z,2019-02-01T01:00:00.0000000Z,2019-02-01T00:00:00.0000000Z,2019-02-01T00:59:44.0000000Z,32.049,34.338460,30.559354,32.112,11136.474940,746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01T01:00:00.0000000Z,2019-02-01T02:00:00.0000000Z,2019-02-01T01:00:12.9630000Z,2019-02-01T01:59:51.7170000Z,31.236,34.338460,30.500000,30.650,42135.142120,2220,...,11136.47494,746.0,14.928251,NaN,NaN,NaN,-3,NaN,NaN,NaN
2,2019-02-01T02:00:00.0000000Z,2019-02-01T03:00:00.0000000Z,2019-02-01T02:00:01.0000000Z,2019-02-01T02:59:50.0000000Z,31.417,33.779000,30.000000,31.615,17561.063430,1158,...,42135.14212,2220.0,18.979794,-4.659713,NaN,NaN,+2,-3,NaN,NaN
3,2019-02-01T03:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T03:00:07.4060000Z,2019-02-01T03:59:54.0000000Z,30.820,33.866176,30.030000,30.790,11725.014970,967,...,17561.06343,1158.0,15.164994,3.099903,-4.659713,NaN,-2,+2,-3,NaN
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T04:59:57.5170000Z,30.780,33.915000,29.927740,30.800,6848.357303,721,...,11725.01497,967.0,12.125145,-2.644173,3.099903,-4.659713,+1,-2,+2,-3


In [8]:
data.columns

Index(['time_period_start', 'time_period_end', 'time_open', 'time_close',
       'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded',
       'trades_count', 'vol_per_count', 'roi', 'price_close_lag1',
       'price_close_lag2', 'price_close_lag3', 'volume_traded_lag1',
       'trades_count_lag1', 'vol_per_count_lag1', 'roi_lag1', 'roi_lag2',
       'roi_lag3', 'cut', 'cut_lag1', 'cut_lag2', 'cut_lag3'],
      dtype='object')

In [9]:
data_ = data.dropna()
data_.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T04:59:57.5170000Z,30.780,33.915000,29.927740,30.800,6848.357303,721,...,11725.014970,967.0,12.125145,-2.644173,3.099903,-4.659713,+1,-2,+2,-3
5,2019-02-01T05:00:00.0000000Z,2019-02-01T06:00:00.0000000Z,2019-02-01T05:00:22.3400000Z,2019-02-01T05:59:55.0000000Z,30.800,33.932000,30.580000,30.800,12514.174980,861,...,6848.357303,721.0,9.498415,0.032473,-2.644173,3.099903,-1,+1,-2,+2
6,2019-02-01T06:00:00.0000000Z,2019-02-01T07:00:00.0000000Z,2019-02-01T06:00:30.7000000Z,2019-02-01T06:59:44.0000000Z,30.810,33.813000,30.094184,33.626,7663.742309,616,...,12514.174980,861.0,14.534466,0.000000,0.032473,-2.644173,+4,-1,+1,-2
7,2019-02-01T07:00:00.0000000Z,2019-02-01T08:00:00.0000000Z,2019-02-01T07:00:12.3660000Z,2019-02-01T07:59:58.0000000Z,30.900,33.560000,30.048215,30.750,13616.153070,1230,...,7663.742309,616.0,12.441140,8.778489,0.000000,0.032473,-4,+4,-1,+1
8,2019-02-01T08:00:00.0000000Z,2019-02-01T09:00:00.0000000Z,2019-02-01T08:00:00.0000000Z,2019-02-01T08:59:34.0000000Z,31.575,34.067559,30.041782,31.230,40162.595770,1834,...,13616.153070,1230.0,11.070043,-8.940958,8.778489,0.000000,+1,-4,+4,-1


In [10]:
y = data_['price_close']
X = data_[['price_close_lag1', 'price_close_lag2', 'price_close_lag3', 
           'roi_lag1', 
           'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [12]:
mod_linear = LinearRegression()
mod_linear.fit(X_train, y_train);

In [13]:
print('coefficients:')
for i in range(len(X.columns)):
    print('{}: {}'.format(X.columns[i], mod_linear.coef_[i]))

coefficients:
price_close_lag1: 1.0285745180166912
price_close_lag2: -0.20661345441085327
price_close_lag3: 0.1767308899766726
roi_lag1: -0.2429167074424031
volume_traded_lag1: 4.684494559296752e-06
trades_count_lag1: -8.19301267652164e-05
vol_per_count_lag1: -0.006153773891308004


In [14]:
y_predicted = mod_linear.predict(X_test)
print('mse:')
print(mean_squared_error(y_test, y_predicted))

mse:
1.8487064058350648


In [15]:
print('accuracy:')
print(mod_linear.score(X_test, y_test))

accuracy:
0.9135586888745447


### clf

In [16]:
# y_clf = data_['cut']
# X_clf = data_[['roi_lag1', 'roi_lag2', 'roi_lag3', 
#                'price_close_lag1', 
#                'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [17]:
y_clf = data_['cut']
X_clf = data_[['roi_lag1',  
               'price_close_lag1', 
               'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [18]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=0)

In [19]:
# random forest

In [20]:
clf_rf = RandomForestClassifier(n_estimators=1000, max_depth=5, bootstrap=True, random_state=0)
clf_rf.fit(X_clf_train, y_clf_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [21]:
print('accuracy:')
print(clf_rf.score(X_clf_test, y_clf_test))

accuracy:
0.4238875878220141


In [22]:
y_clf_predicted = clf_rf.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,106,0,0,0,53,3,1,0
Actual -3,28,0,0,0,7,1,0,0
Actual -2,2,0,0,0,1,0,0,0
Actual -1,0,0,0,0,2,0,0,0
Actual +1,103,0,0,0,64,7,0,1
Actual +2,10,0,0,0,17,8,0,0
Actual +3,0,0,0,0,5,1,0,0
Actual +4,1,0,0,0,1,2,0,3


In [23]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.424      0.         0.         0.         0.42666667 0.36363636
 0.         0.75      ]


In [24]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.65030675 0.         0.         0.         0.36571429 0.22857143
 0.         0.42857143]


In [25]:
# boosting

In [26]:
clf_gb = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, 
                                            max_depth=1, random_state=0)
clf_gb.fit(X_clf_train, y_clf_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [27]:
print('accuracy:')
print(clf_gb.score(X_clf_test, y_clf_test))

accuracy:
0.09860973043597676


In [28]:
y_clf_predicted = clf_gb.predict(X_clf_test)
y_clf_predicted = pd.DataFrame(y_clf_predicted)
y_clf_predicted['cut'] = pd.cut(y_clf_predicted[0], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])

In [29]:
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted['cut']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,135,0,0,0,28,0,0,0
Actual -3,30,0,0,0,6,0,0,0
Actual -2,2,0,0,0,1,0,0,0
Actual -1,1,0,0,0,1,0,0,0
Actual +1,125,1,0,0,48,1,0,0
Actual +2,13,0,0,0,22,0,0,0
Actual +3,1,0,0,0,4,1,0,0
Actual +4,1,0,0,0,3,3,0,0


In [30]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average precision for the 8 intervals is -  [0.43831169 0.         0.         0.         0.42477876 0.
 0.         0.        ]


In [31]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average recall for the 8 intervals is -  [0.82822086 0.         0.         0.         0.27428571 0.
 0.         0.        ]


In [32]:
# Adaboosting

In [33]:
clf_adab = AdaBoostClassifier(n_estimators=1000, learning_rate=1, algorithm='SAMME')
clf_adab.fit(X_clf_train,y_clf_train)

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1,
          n_estimators=1000, random_state=None)

In [34]:
print('accuracy:')
print(clf_adab.score(X_clf_test, y_clf_test))

accuracy:
0.4028103044496487


In [35]:
y_clf_predicted = clf_adab.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,91,1,6,1,53,3,8,0
Actual -3,20,1,4,1,7,2,1,0
Actual -2,2,0,0,0,0,0,1,0
Actual -1,0,0,0,0,2,0,0,0
Actual +1,78,0,7,3,72,6,9,0
Actual +2,11,0,0,0,14,7,3,0
Actual +3,1,0,0,1,0,2,1,1
Actual +4,1,0,0,0,3,3,0,0


In [36]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.44607843 0.5        0.         0.         0.47682119 0.30434783
 0.04347826 0.        ]


In [37]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.55828221 0.02777778 0.         0.         0.41142857 0.2
 0.16666667 0.        ]
